In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="6ZzKpeXkDWaUG2A3KCdo")
project = rf.workspace("fa23ai02-objdetection-2").project("full-cable-synthetic")
dataset = project.version(2).download("voc")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Full-Cable--+--Synthetic-2 in voc:: 100%|██████████| 16243/16243 [00:01<00:00, 10298.88it/s]


In [4]:
%cd ..

/fasterrcnn-pytorch-training-pipeline


In [9]:
!ls

' custom_data'				    inference.py
 LICENSE				    inference_video.py
 README.md				    models
 __init__.py				    notebook_examples
 __pycache__				    onnx_inference_image.py
 _config.yml				    onnx_inference_video.py
 custom_faster_rcnn_training_kaggle.ipynb   outputs
 data					    readme_images
 data_configs				    requirements.txt
 datasets.py				    torch_utils
 docs					    train.py
 eval.py				    utils
 example_test_data			    wandb
 export.py


## Create the Custom Dataset YAML File

The YAML file should contain:
* `TRAIN_DIR_IMAGES`: Path to the training images directory.
* `TRAIN_DIR_LABELS`: Path to the training labels directory containing the XML files. Can be the same as `TRAIN_DIR_IMAGES`.
* `VALID_DIR_IMAGES`: Path to the validation images directory.
* `VALID_DIR_LABELS`: Path to the validation labels directory containing the XML files. Can be the same as `VALID_DIR_IMAGES`.
* `CLASSES`: All the class names in the dataset along with the `__background__` class as the first class.
* `NC`: The number of classes. This should be the number of classes in the dataset + the background class. If the number of classes in the dataset are 7, then `NC` should be 8.
* `SAVE_VALID_PREDICTION_IMAGES`: Whether to save the prediction results from the validation loop or not.

In [2]:
%%writefile data_configs/custom_data.yaml
# Images and labels direcotry should be relative to train.py
TRAIN_DIR_IMAGES: 'custom_data/train'
TRAIN_DIR_LABELS: 'custom_data/train'
VALID_DIR_IMAGES: 'custom_data/valid'
VALID_DIR_LABELS: 'custom_data/valid'

# Class names.
CLASSES: [
    '__background__',
    'Breadboard', 'Button', 'Buzzer', 'Infared_Sensor', 'Keypad', 'LCD', 'LED', 'LED_Matrix', 'Number_LED', 'Photoresistor', 'Potentiometer', 'Remote', 'Resistor', 'Rheostat', 'Servo_Engine', 'Thermistor', 'Tool_Box', 'Tool_Box_Tray', 'UNO_R3', 'USB_Cable', 'Ultrasonic_Sensor', 'Wire'
]

# Number of classes (object classes + 1 for background class in Faster RCNN).
NC: 23

# Whether to save the predictions of the validation set while training.
SAVE_VALID_PREDICTION_IMAGES: True

Writing data_configs/custom_data.yaml


In [3]:
!wandb disabled

W&B disabled.


In [4]:
!export CUDA_VISIBLE_DEVICES=0,1
# python -m torch.distributed.launch --nproc_per_node=2 --use_env train.py --data data_configs/smoke.yaml --epochs 100 --model fasterrcnn_resnet50_fpn --name smoke_training --batch 16

In [6]:
# export CUDA_VISIBLE_DEVICES=0,1
!python -m torch.distributed.launch --nproc_per_node=2 --use_env train.py --data data_configs/custom_data.yaml --epochs 50 --model fasterrcnn_resnet50_fpn_v2 --name custom_training --batch 16 --mosaic 1 -uta

/opt/conda/lib/python3.10/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 0): env://
device cuda
Checking Labels and images...
100%|████████████████████████████████████| 7465/7465 [00:00<00:00, 22948.34it/s]
Checking Labels and images...
100%|█████

## Evaluation

In [7]:
# No verbose mAP.
!python eval.py --weights outputs/training/custom_training/best_model.pth --data data_configs/custom_data.yaml --model fasterrcnn_resnet50_fpn_v2

Checking Labels and images...
100%|███████████████████████████████████████████| 57/57 [00:13<00:00,  4.36it/s]


{'classes': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22], dtype=torch.int32),
 'map': tensor(0.3683),
 'map_50': tensor(0.6061),
 'map_75': tensor(0.4201),
 'map_large': tensor(0.4011),
 'map_medium': tensor(0.4146),
 'map_per_class': tensor(-1.),
 'map_small': tensor(0.1434),
 'mar_1': tensor(0.4002),
 'mar_10': tensor(0.5240),
 'mar_100': tensor(0.5296),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(0.5458),
 'mar_medium': tensor(0.5447),
 'mar_small': tensor(0.2621)}


In [8]:
# Verbose mAP.
!python eval.py --weights outputs/training/custom_training/best_model.pth --data data_configs/custom_data.yaml --model fasterrcnn_resnet50_fpn_v2 --verbose

Checking Labels and images...
100%|███████████████████████████████████████████| 57/57 [00:13<00:00,  4.36it/s]


{'classes': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22], dtype=torch.int32),
 'map': tensor(0.3683),
 'map_50': tensor(0.6061),
 'map_75': tensor(0.4201),
 'map_large': tensor(0.4011),
 'map_medium': tensor(0.4146),
 'map_per_class': tensor([0.4840, 0.3790, 0.4729, 0.5415, 0.4687, 0.5364, 0.2900, 0.2741, 0.3138,
        0.2962, 0.2752, 0.1950, 0.3526, 0.6287, 0.4315, 0.4254, 0.1967, 0.1689,
        0.4054, 0.1930, 0.3716, 0.4017]),
 'map_small': tensor(0.1434),
 'mar_1': tensor(0.4002),
 'mar_10': tensor(0.5240),
 'mar_100': tensor(0.5296),
 'mar_100_per_class': tensor([0.5745, 0.4676, 0.7200, 0.6667, 0.5800, 0.7273, 0.4515, 0.3286, 0.4750,
        0.4071, 0.5032, 0.3500, 0.4810, 0.7200, 0.5600, 0.6500, 0.4364, 0.3727,
        0.6043, 0.4769, 0.5055, 0.5938]),
 'mar_large': tensor(0.5458),
 'mar_medium': tensor(0.54